# Часть 1: аутентификация, работа с API

## Методы аутентификации

| Метод | Что это и как работает | Где используется | Примеры сервисов | Плюсы | Минусы |
|--------|------------------------|------------------|------------------|--------|--------|
| **Session-based аутентификация** | При входе на сервере создается сессия (отправляется POST-запрос с credentials в его теле), а клиент получает `session_id` в составе cookies в ответ. При каждом запросе сервер проверяет эту сессию | Веб-приложения с авторизацией | Wordpress, интернет-магазины | Удобно для пользователей, так как позволяет избежать необходимости авторизовываться при каждом запросе ресурса | Не подходит для API*, требует хранения сессий на сервере |
| **API-ключи** | Клиент получает уникальный ключ (alphanumeric string) и передает его в заголовке запроса или в URL. Такой ключ генерируется как правило один раз при регистрации на сервисе, как правило не имеет срока действия и используется для предоставления общего доступа к ресурсу (т.е. не содержит в себе **scope**) | Открытые API, публичные сервисы | OpenWeather, NASA API, Кинопоиск | Простая реализация, удобство в использовании | Если передавать в URL, можно легко перехватить (например, через реферер)* |
| **Basic-аутентификация** | Логин и пароль кодируются в Base64 (бинарные данные кодируются с помощью 64 ascii-символов) и передаются в заголовке `Authorization: Basic <encoded_credentials>`. | Закрытые API, внутренние сервисы | Примеры в документации REST API, старые API-интерфейсы | Простота использования (не требует сложных механизмов) | Низкая безопасность (логин и пароль могут быть украдены, даже в Base64) |
| **Bearer-аутентификация** | Клиент получает `access_token`, который передается в заголовке `Authorization: Bearer <token>`. Токен действует ограниченное время | API, требующие аутентификации | Spotify API, GitHub API| Безопаснее, чем логин и пароль, можно легко отозвать токен | Токен истекает, его нужно обновлять |
| **OAuth 2.0 (Open Authorization)** | Клиент получает `access_token` через процесс авторизации (например, через редиректы и выдачу разрешений пользователем). | API от имени пользователя | Google API, Facebook API, Twitter API, Spotify API | Позволяет давать доступ к сервису без передачи пароля | Сложен в реализации, требует дополнительных шагов |

**Справка**

1) Session-based аутентификация не подходит для API в основном по следующим причинам:
- API (особенно RESTful) по своей сути без состояния (stateless). Это означает, что каждый запрос к серверу должен **уже** содержать всю необходимую информацию для обработки, включая информацию о пользователе, его аутентификации и авторизации. Этот принцип (в REST) помогает хранить гораздо меньше информации о пользователе и облегчает масштабируемость
- Куки привязаны к конкретной пользовательской сессии на конкретном устройстве (в то время как ключ/токен можно выдать что называется platform-agnostic)
2) Из ограничений API-ключей:
- Ключ генерируется один раз и не истекает --> проблемы с безопасностью
- Ключ не содержит информации о правах доступа конкретного пользователя
- Часто процесс генерации происходит через веб-интерфейс (т.е. закодить сложнее)

### API-ключи

#### Небольшая вставка (`pathlib` vs `urllib`)

В то время как pathlib является модулем, которым стоит пользоваться, когда мы работаем с путями до объектов внутри файловой системы, при работе с адресами URL лучше использовать другие решения (например, urllib)

In [125]:
import pathlib
import urllib
from urllib.parse import urljoin


base_url = "https://somesite.com"
catalogue = "catalogue_name"



# pathlib
print(f"Объединение через pathlib: {pathlib.Path(base_url) / catalogue}")

# urllib
print(f"Объединение через urllib: {urljoin(base_url, catalogue)}")

Объединение через pathlib: https:/somesite.com/catalogue_name
Объединение через urllib: https://somesite.com/catalogue_name


[Небольшой ридинг про urllib](https://arjancodes.com/blog/using-python-urllib-module-for-web-scraping-and-url-parsing/)

Метод `urllib.parse.urlparse` можно использовать для извлечения частей URL-адреса:

In [130]:
from urllib.parse import urlparse

base_url = "https://somesite.com"
catalogue = "catalogue_name"
full_url = urljoin(base_url, catalogue)


parsed_url = urlparse(full_url)
print(f"Full path: {full_url}")
print(f"Scheme: {parsed_url.scheme}")
print(f"Hostname: {parsed_url.hostname}")  # есть ещё netloc, который бы вернул название хоста + порта (можете попробовать с адресом типа http://localhost:8888/lab/tree)
print(f"Path after hostname: {parsed_url.path}")

Full path: https://somesite.com/catalogue_name
Scheme: https
Hostname: somesite.com
Path after hostname: /catalogue_name


Вернёмся к кинопоиску!

---

__Пример: Кинопоиск__:
1) Заходим на сайт с [документацией](https://api.kinopoisk.dev/documentation)
2) Как описано в документации кинопоиска, для официального доступа к их API нужно получить токен через телеграм-бота (ссылка на бота в документации)
3) В данном примере я выбираю бесплатный тариф (на момент подготовки семинара это 200 запросов в сутки)
4) Получаю токен и создаю следующую строку в созданном рядом с данном ноутбуком `.env` файле: `kinopoisk_key=xxx...`

In [132]:
from dotenv import load_dotenv
import os

# все токены/ключи, используемые в данном семинаре, я буду хранить в отдельном .env файле
# процесс получения описан текстом + демонстрируется на семинаре
load_dotenv()

True

In [133]:
kinopoisk_key = os.getenv("kinopoisk_key")

Давайте попробуем с помощью полученного ключа достать какое-то количество фильмов в жанре триллер за 2024 год

In [139]:
import requests

def get_kinopoisk_endpoint(endpoint_name: str, params: dict, headers={}):
    base_url="https://api.kinopoisk.dev/v1/"

    full_url = urljoin(base_url, endpoint_name)
    resp = requests.get(full_url, params=params, headers=headers)

    return resp

In [142]:
endpoint_name = "movie/possible-values-by-field"
params = {"field": "genres.name"}
headers = {"X-API-KEY": kinopoisk_key}

resp = get_kinopoisk_endpoint(endpoint_name, params, headers=headers)  # нам нужно найти триллеры

In [151]:
endpoint_name = "movie"
params = {"genres.name": "триллер", 
         "page": "1", 
         "limit": "250",
         "year": "2024"}
headers = {"X-API-KEY": kinopoisk_key}
resp = get_kinopoisk_endpoint(endpoint_name, params, headers=headers)  # теперь осталось изучить структуру полученного json-объекта

In [154]:
jsonized_response = resp.json()

In [155]:
jsonized_response.keys()

dict_keys(['docs', 'total', 'limit', 'page', 'pages'])

## Про Basic-аутентификацию и Bearer-токены

__Basic-аутентификация__ заключается в передаче в заголовке отдельного параметра `Authorization`, который устанавливается в что-то вроде `Basic <encoded_credentials>`, где `<encoded_credentials>` - это закодированные в base64, например, логин и пароль (или аналоги)

Такой формат аутентификации как правило не используется как самостоятельный, однако в лекции мы видели на примере Spotify API, как с помощью basic-аутентификации мы отправляем `client_id` (аналог логина для приложения) и `client_secret` (аналог пароля), в ответ получая `Bearer-token`

__Bearer-токены__ как правило используется в качестве **элемента** процесса аутентификации, поскольку эти токены должны откуда-то взяться (для их получения, например, нужно сформировать какой-то POST-запрос или проделать набор шагов через веб-интерфейс сервиса). Токены являются как правило набором символов, которые мы тоже передаем в заголовке `Authorization` вместе с запросом. Значение параметра как правило имеет вид `Bearer <token>`, где `<токен>` - какая-то строка

# Часть 2: Selenium

## Введение

__Почему requests + BeautifulSoup не всегда работают?__
- **Requests** и **BeautifulSoup** — отличные инструменты для парсинга статических HTML-страниц. Однако они не могут работать с контентом, который генерируется динамически с помощью JavaScript
- Например, если сайт загружает данные с помощью JavaScript после первоначальной загрузки страницы, запрос через `requests` не получит нужный контент (requests не выполняет JavaScript-код)
- В таких случаях для парсинга (и скрэппинга перед ним) нужно использовать инструменты, которые могут взаимодействовать с браузером, такие как **Selenium** 

__Что такое Selenium?__
- [**Selenium**](https://www.selenium.dev/) — это библиотека для автоматизации браузеров (открытие страниц, клики, ввод текста, скроллы и т.д.)
- Selenium может управлять реальными браузерами, такими как Chrome, Firefox, и другие, что позволяет работать с динамическим контентом, который генерируется с помощью JavaScript

__Как работает WebDriver (Chromedriver, Geckodriver)__
- **WebDriver** — это интерфейс для взаимодействия с браузером.
- **Chromedriver** — это инструмент, который позволяет Selenium управлять браузером Google Chrome.
- **Geckodriver** — аналог для браузера Mozilla Firefox.
- Эти драйвера необходимы для того, чтобы Selenium мог взаимодействовать с конкретным браузером.

In [62]:
# раскомментируйте строчку ниже для установки Selenium'а
# !pip install selenium

__Минимальный пример работы:__

In [4]:
from selenium import webdriver  # импортируем вебдрайвер
import time
# импортируем объект для доступа к различным локаторам 
# примеры таких локаторов в документации https://www.selenium.dev/selenium/docs/api/py/webdriver/selenium.webdriver.common.by.html
from selenium.webdriver.common.by import By

In [173]:
driver = webdriver.Chrome()  # инициализируем веб-драйвер

driver.get("https://student.itmo.ru/ru/dormitory/")  # переходим на веб-страницу с общежитями ИТМО

In [177]:
cat_page_navig = driver.find_element(by=By.CLASS_NAME, value="catalog-page__navigation-link.icon-arrow-left")  # выбираем объекты по CSS-селекторуtime.sleep(2)  # здесь ожидание вставлено для удобства демонстрации

In [184]:
time.sleep(2)
cat_page_navig.click()

In [185]:
driver.quit()  # закрываем драйвер; любые попытки взаимодействия с драйвером после этой строки приведут к выбросу ошибки

**Примечание**

Если хочется получить доступ ко всему HTML-коду веб-страницы, можно посмотреть в атрибут `driver.page_source`

## Наиболее частые локаторы

**Локаторы** – это способы поиска элементов на веб-странице. Selenium предоставляет несколько типов локаторов, которые позволяют нам идентифицировать нужные элементы, чтобы взаимодействовать с ними (например, заполнять формы, кликать по кнопкам или извлекать текст).

В то время как Selenium из коробки предлагает работу с несколькими локаторами, для нас наиболее частыми скорее всего будут следующие:

- `TAG_NAME` – поиск элемента по его HTML-тегу
- `CLASS_NAME` – поиск элемента по его классу
- `CSS_SELECTOR` – поиск элемента с использованием CSS-селекторов

Давайте рассмотрим каждый из этих локаторов на примере вот такого HTML-документа:

```
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>HTML Locators Example</title>
</head>
<body>
  <h1>Welcome to the Locators Demo</h1>
  
  <div id="container">
    <p class="description">This is a sample paragraph.</p>
    <p id="intro">Intro paragraph.</p>
    
    <ul class="list">
      <li><a href="https://example.com">Example Link 1</a></li>
      <li><a href="https://example2.com">Example Link 2</a></li>
      <li><a href="https://example3.com">Example Link 3</a></li>
    </ul>
    
    <button id="submit-button">Submit</button>
    
    <div class="form-group">
      <label for="username">Username:</label>
      <input type="text" id="username" name="username" value="john_doe">
    </div>
    
    <form id="form1">
      <input type="checkbox" id="terms" name="terms"> Accept Terms and Conditions
      <input type="submit" value="Submit">
    </form>
  </div>
  
  <footer>
    <p>Contact us at <a href="mailto:support@example.com">support@example.com</a></p>
  </footer>
</body>
</html>
```

В отличие от `bs4`, для рассмотрения такой страницы через Selenium, нам нужно как-то открыть её в браузере. Для сохраним веб-страницу локально (в текстовый файл) и откроем его через Selenium. Далее мы будем честно извлекать веб-страницы по не локальным URL-адресам

In [186]:
selenium_html_example = '''
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>HTML Locators Example</title>
</head>
<body>
  <h1>Welcome to the Locators Demo</h1>
  
  <div id="container">
    <p class="description">This is a sample paragraph.</p>
    <p id="intro">Intro paragraph.</p>
    
    <ul class="list">
      <li><a href="https://example.com">Example Link 1</a></li>
      <li><a href="https://example2.com">Example Link 2</a></li>
      <li><a href="https://example3.com">Example Link 3</a></li>
    </ul>
    
    <button id="submit-button">Submit</button>
    
    <div class="form-group">
      <label for="username">Username:</label>
      <input type="text" id="username" name="username" value="john_doe">
    </div>
    
    <form id="form1">
      <input type="checkbox" id="terms" name="terms"> Accept Terms and Conditions
      <input type="submit" value="Submit">
    </form>
  </div>
  
  <footer>
    <p>Contact us at <a href="mailto:support@example.com">support@example.com</a></p>
  </footer>
</body>
</html>
'''

with open("selenium_html_example.html", "w") as f:
    f.write(selenium_html_example)

In [60]:
from pathlib import Path

# Получаем абсолютный путь к файлу
html_file_path = Path("selenium_html_example.html").resolve()

# Открываем файл в браузере
driver = webdriver.Chrome()
driver.get(f"file://{html_file_path}")

#### Локатор TAG_NAME

Этот метод позволяет находить элементы по их HTML-тегу. Например, чтобы найти все \<p>-поля на странице:

In [188]:
driver = webdriver.Chrome()
driver.get(f"file://{html_file_path}")

In [189]:
inputs = driver.find_elements(by=By.TAG_NAME, value="p")

In [200]:
# мы можем вывести найденный элемент вместе с его разметкой через .get_attribute("outerHTML")
index_ = 2

found_input_0 = inputs[index_].get_attribute("outerHTML")
print(f"Found p element: {found_input_0}")

# или достать какую-то информацию из найденного тега (например, текст)
print(f"Found <p> tag contains: {inputs[index_].text}")

Found p element: <p>Contact us at <a href="mailto:support@example.com">support@example.com</a></p>
Found <p> tag contains: Contact us at support@example.com


In [201]:
driver.quit()

#### Локатор CLASS_NAME

Этот метод ищет элементы по их CSS-классу. Например, найдем все поля ввода с классом `list` (такой всего один):

In [202]:
driver = webdriver.Chrome()
driver.get(f"file://{html_file_path}")

In [203]:
# Находим все элементы с классом "list"
list_field = driver.find_element(By.CLASS_NAME, "list")


In [209]:
# Далее достанем первый элемент списка
li_tags = list_field.find_elements(By.TAG_NAME, "li")

# И выведем текст первого элемента списка
print(f"First element's text: {li_tags[0].text}")

First element's text: Example Link 1


#### Локатор CSS_SELECTOR

`CSS-локаторы` позволяют искать элементы на основе правил, аналогичных тем, что используются в CSS-стилях. Это мощный инструмент для навигации по DOM-структуре.

`DOM` – это структура HTML-документа в виде дерева объектов, где каждый элемент страницы представлен как узел (node). Браузер строит DOM после загрузки страницы, и JavaScript (или Selenium) может взаимодействовать с ним, изменяя содержимое или структуру.

Подробнее про DOM [здесь](https://blog.skillfactory.ru/glossary/dom/)

__CSS__ (Cascading Style Sheets / Каскадные таблицы стилей) используется для стилизации HTML-страниц. Стили как правило задаются в отдельном файле, хотя могут быть и явно заданы внутри HTML-кода

Пример (этот код применяет стили к любому элементу с class="information"):

```
.information {
  background-color: white;
  color: black;
  padding: 10px;
}
```

__CSS-селектор__ — это выражение, используемое для выбора HTML-элементов на веб-странице на основе их тегов, классов, идентификаторов, атрибутов и структуры документа. В контексте Selenium CSS-селекторы позволяют находить элементы на странице для взаимодействия (клика, ввода текста, проверки свойств и т.д.)

Посмотрим на примеры поиска по CSS-селекторам!

In [61]:
driver = webdriver.Chrome()
driver.get(f"file://{html_file_path}")

In [211]:
# 1. Поиск по ID
# Поиск элемента по ID (например, элемент с ID "container")
element_by_id = driver.find_element(By.CSS_SELECTOR, "#container")
print("Element by ID:", element_by_id.text)

Element by ID: This is a sample paragraph.
Intro paragraph.
Example Link 1
Example Link 2
Example Link 3
Submit
Username:
Accept Terms and Conditions


In [213]:
# 2. Поиск по классу
# Поиск элемента по классу (например, элемент с классом "description")
element_by_class = driver.find_element(By.CSS_SELECTOR, ".description")
print("Element by Class:", element_by_class.text)

Element by Class: This is a sample paragraph.


In [217]:
# 3. Поиск по атрибуту
# Поиск элемента по атрибуту (например, <input> с атрибутом name='username')
input_element = driver.find_element(By.CSS_SELECTOR, "input[name='username']")
print("Input Element by Attribute:", input_element.get_attribute('value'))

Input Element by Attribute: john_doe


In [218]:
# 4. Поиск по тегу
# Поиск первого элемента <p> на странице
first_paragraph = driver.find_element(By.CSS_SELECTOR, "p")
print("First Paragraph:", first_paragraph.text)

First Paragraph: This is a sample paragraph.


```
<ul class="list">
    <li><a href="https://example.com">Example Link 1</a></li>
```

In [220]:
# 5. Поиск дочернего элемента
# Поиск элемента <a> внутри элемента <li> (первый элемент списка)
link_in_list = driver.find_element(By.CSS_SELECTOR, "ul.list li a")
print("First Link in List:", link_in_list.text)

First Link in List: Example Link 1


**Справка**

Синтаксис "ul.list li a" — это CSS-селектор, который используется для поиска дочерних элементов в иерархической структуре HTML. Давайте посмотрим поближе:

- `ul.list`: Это означает, что мы ищем элемент \<ul> (неупорядоченный список), у которого есть класс list. Таким образом, мы ищем `<ul class="list">`.
- `li`: После этого у нас идет тег \<li>. Этот селектор ищет все элементы списка \<li>, которые являются дочерними элементами внутри найденного элемента `<ul class="list">`.
- `a`: Наконец, это означает, что мы ищем элементы \<a> внутри каждого элемента \<li>. Таким образом, этот селектор будет искать все ссылки \<a>, которые находятся внутри каждого элемента списка.

__Важно:__

Этот селектор не обязательно ищет прямого потомка на каждом уровне. Он будет искать все элементы, которые соответствуют цепочке **ul.list → li → a**. 

То есть \<a> может быть не прямым потомком \<li>, а любым потомком внутри, что важно для вложенных структур

In [67]:
# 6. Поиск с псевдоклассом
# Поиск первого элемента <li> в списке
first_list_item = driver.find_element(By.CSS_SELECTOR, "ul.list li:first-child")
print("First List Item:", first_list_item.text)

First List Item: Example Link 1


Подробнее про псевдоклассы с примерами можно посмотреть [здесь](https://developer.mozilla.org/ru/docs/Web/CSS/Pseudo-classes)

## Работа с динамическим контентом, важное

Перед тем как мы попрактикуемся в веб-скрэппинге с использованием Selenium, нам нужно разобрать несколько моментов

Веб-страницы могут загружать контент не сразу при открытии, а динамически, в зависимости от действий пользователя или запроса к 
серверу. Это делается для улучшения производительности и удобства взаимодействия с сайтом.

Примеры динамической подгрузки контента:

- Лента соцсетей загружается по мере прокрутки (например, лента ВК)
- Подтягивание данных при вводе (автодополнение в поиске, например, гугл)
- Кнопка "Показать ещё" загружает дополнительные товары в интернет-магазине
- Комментарии под статьями появляются только после клика на "Открыть комментарии" (без редиректа на новую веб-страницу)


__Что такое AJAX-запросы?__

__AJAX (Asynchronous JavaScript and XML)__ — это технология, позволяющая веб-странице запрашивать данные с сервера без полной перезагрузки страницы

Как работает AJAX:

- Пользователь выполняет действие (например, кликает на кнопку "Загрузить ещё")
- JavaScript отправляет асинхронный HTTP-запрос к серверу
- Сервер обрабатывает запрос и отправляет данные (обычно в формате JSON)
- JavaScript обновляет страницу, добавляя полученные данные, без её перезагрузки

При работе с динамическим контентом для нас важны два момента:
- Триггер подгрузки элементов (скролл / нажатие на кнопку / ввод текста / что-то ещё)
- Ожидание подгрузки элемента/элементов

Поговорим про ожидание и перейдем к практике!

#### Про ожидание ответа

При работе с Selenium браузер не всегда загружает все элементы сразу. 
Иногда контент подгружается асинхронно (например, через AJAX), что делает стандартный поиск элементов ненадежным – Selenium может пытаться найти элемент до того, как он появился на странице.

Чтобы избежать ошибок типа `NoSuchElementException` и `StaleElementReferenceException` (когда-то найденный элемент больше не доступен), используется механизм ожиданий (Waits)

__Виды ожиданий в Selenium__

В Selenium есть три типа ожиданий:
- **Неявные ожидания (Implicit Wait)**. Глобально ждут появления любого элемента перед выполнением следующей команды
- **Явные ожидания (Explicit Wait)**. Ожидают конкретное условие (например, появление элемента) в течение заданного времени
- **Принудительная задержка (time.sleep)**. Останавливает выполнение кода на фиксированное время (неэффективно и нежелательно)

Подробнее про ожидания в [документации](https://www.selenium.dev/documentation/webdriver/waits/)

__Пример Implicit ожидания:__

In [ ]:
# пример не сработает

driver = webdriver.Chrome()

driver.get("https://itmo.ru/")
burger_menu = driver.find_elements(by=By.CSS_SELECTOR, value=".NavItem_container__fbdal")[2]
burger_menu.click()
driver.find_element(by=By.CSS_SELECTOR, value=".Accordion_heading__link__FYF1W").click()

In [226]:
driver.quit()

In [ ]:
# пример сработает

driver = webdriver.Chrome()
driver.implicitly_wait(5)  # глобальная настройка ожидания до 5 секунд при каждом вызове find_elements

driver.get("https://itmo.ru/")
burger_menu = driver.find_elements(by=By.CSS_SELECTOR, value=".NavItem_container__fbdal")[2]
burger_menu.click()
driver.find_element(by=By.CSS_SELECTOR, value=".Accordion_heading__link__FYF1W").click()

__Пример Explicit ожидания__

In [8]:
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
driver = webdriver.Chrome()
driver.get("https://itmo.ru/")

burger_menu = driver.find_elements(by=By.CSS_SELECTOR, value=".NavItem_container__fbdal")[2]
burger_menu.click()

wait = WebDriverWait(driver, timeout=3)
button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".Accordion_heading__link__FYF1W")))

button.click()

__Частые explicit условия (EC.***)__

| Условие                                   | Описание                                         |
|-------------------------------------------|-------------------------------------------------|
| `element_to_be_clickable((By., ""))`      | Элемент стал кликабельным                      |
| `presence_of_element_located((By., ""))`  | Элемент появился в DOM (но не обязательно видим) |
| `visibility_of_element_located((By., ""))`| Элемент появился и стал видимым                 |
| `text_to_be_present_in_element((By., ""), "текст")` | В элементе появился нужный текст |


# Часть 3: Практика Selenium

## Кейс 1: Lamoda и "показать ещё"

In [2]:
driver = webdriver.Chrome()
driver.get("https://www.lamoda.ru/c/151/shoes-muzhskie-botinki/?sitelink=topmenuM&l=2&page=1")

In [232]:
button = driver.find_element(by=By.CLASS_NAME, value="x-button.x-button_secondaryFilledWeb7184.x-button_40._showMore_z1yqr_2")

In [234]:
time.sleep(2)
button.click()

## Кейс 2: Yandex.Market и динамическая подгрузка через скролл

1. Зайдем на яндекс.маркет
2. Впишем в форму поиска название какого-то товара
3. Кликнем "Найти"
4. Проскроллим до конца

In [ ]:
driver = webdriver.Chrome()
driver.get("https://market.yandex.ru/")

In [ ]:
driver_options = webdriver.ChromeOptions()
driver_options.add_argument("--disable-blink-feature=AutomationControlled")
driver_options.add_argument("user-agent=Mozilla/5.0 (X11; Linux x86_64; rv:133.0) Gecko/20100101 Firefox/133.0")

driver = webdriver.Chrome(options=driver_options)
driver.get('https://market.yandex.ru/')

In [238]:
search_field = driver.find_element(By.ID, "header-search")
time.sleep(2)
search_field.click()

In [239]:
search_field.send_keys("чайник фарфоровый")

In [240]:
find_button = driver.find_element(By.CSS_SELECTOR, "button[type='submit']")
time.sleep(2)
find_button.click()

[Про скроллинг](https://www.selenium.dev/documentation/webdriver/actions_api/wheel/)

In [11]:
from selenium.webdriver import ActionChains

In [219]:
time.sleep(2)
product_containers = driver.find_elements(by=By.CLASS_NAME, value="cia-cs.Gqfzd")
ActionChains(driver).scroll_to_element(product_containers[-1]).perform()

## Кейс 3 (продолжение): Яндекс.Маркет и переключение между окнами

In [12]:
driver_options = webdriver.ChromeOptions()
driver_options.add_argument("--disable-blink-feature=AutomationControlled")
driver_options.add_argument("user-agent=Mozilla/5.0 (X11; Linux x86_64; rv:133.0) Gecko/20100101 Firefox/133.0")

driver = webdriver.Chrome(options=driver_options)
driver.get('https://market.yandex.ru/')

In [14]:
# ждём, пока не появится всплывающее окно, предлагающее получить какой-то бонус (оно перекрывает доступ ко всей странице)
wait = WebDriverWait(driver, timeout=10)  # будем ждать, пока окно не появится, в течение 10 секунд
popup_window_to_close = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "._2veCe.cia-vs")))

location = popup_window_to_close.location  # Словарь с координатами {"x": <x-координата>, "y": <y-координата>}
size = popup_window_to_close.size  # Словарь с размерами {"width": <ширина>, "height": <высота>}

# Координаты верхнего левого угла
x, y = location['x'], location['y']
# +10% вверх от размера рекламного блока 
to_top_10 = int(size['height']*0.1)

# Теперь можно использовать эти координаты для выполнения цепочки действий:
# передвинуть мышку за пределы всплывающего блока -> нажать клик по координате (x, y-to_top_10) 
actions = ActionChains(driver)
actions.move_by_offset(x, y-to_top_10).click().perform()

In [15]:
# находим поисковую строку
search_field = driver.find_element(By.ID, "header-search")
search_field.click()  

# вбиваем поисковый запрос "чайник фарфоровый"
search_field.send_keys("чайник фарфоровый")

# находим кнопку "Найти", чтобы отправить поисковый запрос
find_button = driver.find_element(By.CSS_SELECTOR, "button[type='submit']")
find_button.click()

# находим список всех товаров (здесь - включая промо-товары)
product_containers = driver.find_elements(by=By.CLASS_NAME, value="_2rw4E._2O5qi")
# кликаем на первый товар
first_item = product_containers[0]
# находим изображение (thumbnail) первого товара
item_image = first_item.find_element(by=By.CSS_SELECTOR, value='div[class="cia-vs"][data-baobab-name="pictureGallery"]')


In [32]:
# перед открытием товара (он откроется в другой вкладке) сохраним адрес текущей открытой вкладки
all_items_window = driver.current_window_handle
# теперь кликаем по thumbnail'у товара (откроется новая вкладка)
item_image.click()

[Про работу с окнами](https://www.selenium.dev/documentation/webdriver/interactions/windows/)

In [33]:
# получим список открытых на данный момент вкладок
windows = driver.window_handles
print(windows)

['F4E206CE20292B17E7EEC1B0CACB0B74', '010123DFD327FFC5DA3AAAE0992C463C']


In [36]:
# убедимся в том, что мы сейчас находимся в новой вкладке 
driver.current_window_handle != all_items_window

False

In [37]:
# если выражений оценено как False, надо переключиться на новую вкладку
driver.switch_to.window(windows[-1])  # переключиться на последнюю вкладку

In [38]:
# убедимся в том, что мы сейчас находимся в новой вкладке (теперь должно быть True)
driver.current_window_handle != all_items_window

True

# Вставка про установку Selenium driver

Для того, чтобы мы могли инициализировать виртуальный браузер, необходим драйвер (считайте, что это объект-интерфейс) для доступа к этому самому браузеру

Если инициализация драйвера как `driver = webdriver.Chrome()` приводит к ошибке типа Unable to Locate Driver, то скорее всего драйвер не установлен

Если он установлен, вы можете найти информацию о его версии, а также узнать, где именно он установлен, выполнив скрипт ниже

In [40]:
from selenium import webdriver

driver = webdriver.Chrome()
version = driver.capabilities['browserVersion']  # получить доступ к версии браузера
chromedriver_version = driver.capabilities['chrome']['chromedriverVersion']  # получить доступ к версии драйвера
print(f"Chrome Browser Version: {version}")
print(f"ChromeDriver Version: {chromedriver_version}")
print(f"ChromeDriver Path: {driver.service.path}")
driver.quit()

Chrome Browser Version: 129.0.6668.89
ChromeDriver Version: 129.0.6668.100 (cf58cba358d31ce285c1970a79a9411d0fb381a5-refs/branch-heads/6668@{#1704})
ChromeDriver Path: /home/kiri/.cache/selenium/chromedriver/linux64/129.0.6668.100/chromedriver


Обратите внимание на то, что версия драйвера и версия браузера должны совпадать по первой чиселке (в примере выше это 129)

Если драйвер не установлен, его можно установить двумя способами:
- Скачать нужный драйвер и добавить путь до него в переменную PATH
- Использовать утилиту `webdriver-manager`, которая возьмёт установку (и трекинг обновлений) драйвера на себя

### Способ 1

1. Устанавливаем драйвер для нужного браузера из [этого списка](https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location/#download-the-driver)

**Примечания**
- В списке по ссылке выше неверная ссылка на драйвера хрома. Драйвер для хрома можно скачать по [этой ссылке](https://googlechromelabs.github.io/chrome-for-testing/#stable) - берёте из канала Stable под вашу ОС.
- Версия драйвера должна совпадать с вашей версией браузера (например, хрома). Ссылка выше ведет на страницу с последней версией драйвера. Если вам нужна другая версия, вы можете найти её среди [этих эндпоинтов](https://googlechromelabs.github.io/chrome-for-testing/known-good-versions-with-downloads.json)

2. После скачивания драйвера распаковываем архив с ним и добавляем путь до вашего драйвера в переменную PATH ([пример здесь](https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location/)). Путь должен заканчиваться на `chromedriver.exe` для Windows и `chromedriver` на Linux/MacOS

Можно либо поместить веб-драйвер внутрь вашего проекта (в целом, тогда и в переменную PATH класть не нужно, но при инициализации драйвера нужно будет каждый раз указывать до него путь), либо поместить в какое-то типичное глобальное место, типа:
- `C:\Program Files\chromedriver\` на Windows
- `usr/local/bin` на Linux/MacOS 

### Способ 2

1. Устанавливаем нужный драйвер для нужного браузера, как в способе выше
2. Устанавливаем пакет `webdriver-manager` (`pip install webdriver-manager`)\
3. Инициализируем драйвер следующим образом (на примере хрома):


In [55]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

In [56]:
print(f"Path to the cache: {ChromeDriverManager().install()}")

Path to the cache: /home/kiri/.wdm/drivers/chromedriver/linux64/129.0.6668.100/chromedriver-linux64/chromedriver


Такой способ установки (и дальнейшего использования вебдрайвера) основан на том, что пакет `webdriver-manager` берёт на себя:
- установку нужной версии веб-драйвера
- кэширование нужной версии (драйвер ставится в `~/.wdm/` на Linux/macOS и в `C:\Users\YourUsername\.wdm\` на Windows)

После установки можете либо инициализировать драйвер всегда, как выше (если драйвер уже установлен, повторная установка не требуется), либо сохранить путь до драйвера как системную переменную и вставлять её как путь, будет что-то типа `driver = webdriver.Chrome(chromedriver_path)`. Но смысла в этом мало, поскольку проще уж скачать драйвер и закинуть его в `PATH`, как в первом способе

### UPD. Способ 3

Утилита `chromedriver-autoinstaller` (для установки `pip install chromedriver-autoinstaller`) берёт на себя установку драйвера хром и помещения его в PATH (по крайней мере так заявлено)

Для установки драйвера хром можно выполнить код ниже (скопировано из pypi-страницы утилиты)

In [58]:
from selenium import webdriver
import chromedriver_autoinstaller


chromedriver_autoinstaller.install()  # Check if the current version of chromedriver exists
                                      # and if it doesn't exist, download it automatically,
                                      # then add chromedriver to path

driver = webdriver.Chrome()